In [1]:
from typing import List, Dict
import plotly.graph_objects as go

In [2]:
class Collatz:
    """
    A class to generate, analyse, and visualise generalised Collatz sequences.

    This class allows for the exploration of Collatz-like sequences with a customisable
    multiplicative factor (n_factor) for odd numbers in the sequence.

    Attributes:
        n_factor (int): The multiplicative factor used when n is odd (default is 3).
        sequences (Dict[int, List[int]]): Stores generated sequences keyed by start number.
        metrics (Dict[int, Dict[str, int]]): Stores calculated metrics for each sequence.
    """

    def __init__(self, n_factor: int = 3) -> None:
        """
        Initialise the Collatz object with a specified n_factor.

        Args:
            n_factor (int, optional): The multiplicative factor for odd numbers. Defaults to 3.
        """
        self.n_factor: int = n_factor
        self.sequences: Dict[int, List[int]] = {}
        self.metrics: Dict[int, Dict[str, int]] = {}

    def generate_sequence(self, start_number: int) -> List[int]:
        """
        Generate a Collatz-like sequence for a given starting number.

        This method applies the generalised Collatz rules:
        - If n is even, the next number is n/2
        - If n is odd, the next number is (n_factor * n) + 1

        Args:
            start_number (int): The number to start the sequence from.

        Returns:
            List[int]: The generated sequence.

        Raises:
            ValueError: If start_number is less than 1.
        """
        if start_number < 1:
            raise ValueError("Start number must be a positive integer.")
        
        n = start_number
        sequence = [n]
        
        while n != 1:
            if n % 2 == 0:
                n //= 2
            else:
                n = self.n_factor * n + 1
            sequence.append(n)
        
        self.sequences[start_number] = sequence
        self._calculate_metrics(start_number)
        return sequence
    
    def _calculate_metrics(self, start_number: int) -> None:
        """Calculate and store metrics for a given sequence."""
        sequence = self.sequences[start_number]
        self.metrics[start_number] = {
            "length": len(sequence),
            "max_value": max(sequence)
        }

    def calculate_metrics(self, start_number: int) -> Dict[str, int]:
        """
        Calculate various metrics for the sequence of a given starting number.

        Metrics calculated:
        - length: Total number of elements in the sequence
        - max_value: Maximum value reached in the sequence
        - steps_to_one: Number of steps required to reach 1

        Args:
            start_number (int): The starting number of the sequence to analyse.

        Returns:
            Dict[str, int]: A dictionary containing the calculated metrics.
        """
        if start_number not in self.sequences:
            self.generate_sequence(start_number)
        
        sequence = self.sequences[start_number]
        metrics = {
            "length": len(sequence),
            "max_value": max(sequence),
            "steps_to_one": len(sequence) - 1,
        }
        
        self.metrics[start_number] = metrics
        return metrics


    def visualise(self, start_numbers: List[int], common_origin: bool = False) -> None:
        """
        Visualize the Collatz-like sequences for multiple starting numbers.

        Args:
            start_numbers (List[int]): A list of starting numbers to visualize.
            common_origin (bool): If True, shift all sequences to start at x=0. Default is False.

        Note:
            This method will automatically generate any sequences that haven't
            been previously calculated.
        """
        fig = go.Figure()

        for start_number in start_numbers:
            if start_number not in self.sequences:
                self.generate_sequence(start_number)
            
            sequence = self.sequences[start_number]
            
            if common_origin:
                x_values = list(range(len(sequence)))
            else:
                x_values = list(range(self.metrics[start_number]["length"]))
            
            fig.add_trace(go.Scatter(
                x=x_values,
                y=sequence,
                mode='lines+markers',
                name=f"Start {start_number}"
            ))
        
        title = f"Generalised Collatz Sequences (n_factor = {self.n_factor})"
        if common_origin:
            title += " (Common Origin)"
        
        fig.update_layout(
            title=title,
            xaxis_title="Step" if not common_origin else "Step (Shifted)",
            yaxis_title="Value"
        )
        
        fig.show()


    def __str__(self) -> str:
        """
        Provides a concise string representation of the Collatz object.
        """
        num_sequences = len(self.sequences)
        if num_sequences == 0:
            return f"Collatz(n_factor={self.n_factor}, no sequences generated)"
        
        sample_start = next(iter(self.sequences))
        sample_sequence = self.sequences[sample_start]
        sample_metrics = self.metrics[sample_start]
        
        return (f"Collatz(n_factor={self.n_factor}, {num_sequences} sequence(s) generated, "
                f"e.g., {sample_start} -> {self._format_sequence(sample_sequence)} "
                f"(length: {sample_metrics['length']}, max: {sample_metrics['max_value']})")

    def __repr__(self) -> str:
        """
        Provides a detailed string representation of the Collatz object.
        """
        sequences_repr = ", ".join(
            f"{start}: {self._format_sequence(seq)} "
            f"(length: {self.metrics[start]['length']}, max: {self.metrics[start]['max_value']})"
            for start, seq in self.sequences.items()
        )
        return f"Collatz(n_factor={self.n_factor}, sequences={{{sequences_repr}}})"

    def _format_sequence(self, sequence: List[int], max_length: int = 5) -> str:
        """
        Helper method to format a sequence for display.
        """
        if len(sequence) <= max_length:
            return str(sequence)
        return f"[{', '.join(map(str, sequence[:max_length-1]))}, ..., {sequence[-1]}]"

Collatz - standard (n_factor = 3)

In [3]:
collatz = Collatz()  # Uses default n_factor = 3


In [4]:
collatz

Collatz(n_factor=3, sequences={})

In [5]:
print(collatz.generate_sequence(19))

[19, 58, 29, 88, 44, 22, 11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1]


In [6]:
collatz

Collatz(n_factor=3, sequences={19: [19, 58, 29, 88, ..., 1] (length: 21, max: 88)})

In [7]:
n_vis = range(1, 100)

In [8]:
collatz.visualise(n_vis)

In [9]:
collatz.visualise(n_vis, common_origin=True)

In [10]:
# Example with a different n_factor

# Seems that n_factor != 3 is not straightforward

# collatz2 = Collatz(n_factor=2)
# collatz2.generate_sequence(2)
# collatz2.generate_sequence(5)
# collatz6.visualise([2, 5])

In [13]:
collatz1 = Collatz(n_factor=1)
collatz1.generate_sequence(19)
collatz1.generate_sequence(27)
collatz1.visualise(n_vis)